In [27]:
import os
import random
import pickle
import networkx as nx
import math

In [28]:
def load_random_graph(graphs_folder):
    graph_files = [f for f in os.listdir(graphs_folder) if f.endswith('.pkl')]
    random_graph_file = random.choice(graph_files)
    with open(os.path.join(graphs_folder, random_graph_file), 'rb') as f:
        G = pickle.load(f)
    return G, random_graph_file

# Putanja do foldera sa grafovima
graphs_folder = 'grafovi'

# Učitavanje grafa
G_global, graph_file_name = load_random_graph(graphs_folder)
# G_info = nx.info(G)
num_nodes = G_global.number_of_nodes()

# G_info, num_nodes, graph_file_name
num_nodes, graph_file_name

(78, 'graf_42.pkl')

In [29]:
def solution_quality(solution):
    return len(solution.nodes())

In [30]:
def initialize_solution(G, k):
    while True:
        potential_solution = G.copy()
        nodes_to_remove = random.sample(potential_solution.nodes(), k)
        potential_solution.remove_nodes_from(nodes_to_remove)
        if is_k_connected(potential_solution, k):
            return potential_solution

In [31]:
def local_search(G, k, current_solution):
    # Ova funkcija treba da smanji broj čvorova u rešenju, ako je to moguće
    # Proverava da li može da ukloni bilo koji čvor a da i dalje ostane k-vezan
    print("Broj cvorova na ulasku u local_search() ")
    print(current_solution.number_of_nodes())
    temp_solution = current_solution.copy()
    nodes = list(current_solution.nodes())
    random.shuffle(nodes)
    for node in nodes:
        temp_solution.remove_node(node)
        if not is_k_connected(temp_solution, k):
            temp_solution.add_node(node)
    print("Broj cvorova na izlasku iz local_search() ")
    print(temp_solution.number_of_nodes())
    return temp_solution

In [32]:
def is_k_connected(G, k):
    if G.number_of_nodes() < k:
        return False
    else:
        return nx.node_connectivity(G) >= k


In [44]:
def shaking(G, k, current_solution):
    modified_solution = current_solution.copy()

    if random.choice([True, False]):
        # Ukloni slučajno odabrane čvorove, ali ne manje od k
        potential_nodes_to_remove = list(modified_solution.nodes())
        # Računanje težina na osnovu broja suseda
        weights_to_remove = [G.degree(node) for node in potential_nodes_to_remove]
        # Provera da li je k manji od broja potencijalnih čvorova za uklanjanje
        if k > len(potential_nodes_to_remove):
            k = len(potential_nodes_to_remove)
        # Izbor čvorova za uklanjanje na osnovu težina
        nodes_to_remove = random.choices(potential_nodes_to_remove, weights=weights_to_remove, k=k)
        modified_solution.remove_nodes_from(nodes_to_remove)
    else:
        # Dodaj slučajno odabrane čvorove iz originalnog grafa
        potential_nodes = list(set(G.nodes()) - set(modified_solution.nodes()))
        # Računanje težina na osnovu broja suseda
        weights = [G.degree(node) for node in potential_nodes]
        # Izbor čvorova na osnovu težina
        nodes_to_add = random.choices(potential_nodes, weights=weights, k=math.floor(len(potential_nodes)/2))
        modified_solution.add_nodes_from(nodes_to_add)
        # Dodaj i odgovarajuće ivice
        for node in nodes_to_add:
            for neighbor in G.neighbors(node):
                if neighbor in modified_solution:
                    modified_solution.add_edge(node, neighbor)

    # Provera da li je i dalje k-vezan
    print(f'shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: {modified_solution.number_of_nodes()}')
    modified_solution = remove_nodes_with_less_than_k_neighbors_optimized(modified_solution, k)
    print(f'shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: {modified_solution.number_of_nodes()}')
    if is_k_connected(modified_solution, k):
        print("shaking: modifikovano rešenje je prihvatljivo")
        return modified_solution
    else:
        print("shaking: modifikovano rešenje nije prihvatljivo")
        return current_solution


In [45]:
def remove_nodes_with_less_than_k_neighbors_optimized(G, k):
    changed = True
    while changed:
        changed = False
        nodes_to_remove = [node for node in G.nodes if G.degree(node) < k]
        if nodes_to_remove:
            G.remove_nodes_from(nodes_to_remove)
            changed = True
    return G

In [46]:
from time import perf_counter

In [47]:
def vns_algorithm(G, k, max_iter=5):
    # Inicijalizacija
    # current_solution = initialize_solution(G, k)
    # print(current_solution.number_of_nodes())
    start_time = perf_counter()
    G = remove_nodes_with_less_than_k_neighbors_optimized(G, k)
    best_solution = G.copy()
    print(f'vns_algorithm: broj cvorova na pocetku: {G.number_of_nodes()}')
    while perf_counter() - start_time < 2*60:
        # Shaking
        shaken_solution = shaking(G, k, best_solution)
        # Local search
        new_solution = local_search(G, k, shaken_solution)
        
        # Ažuriranje trenutnog rešenja ako je novo rešenje bolje
        if solution_quality(new_solution) < solution_quality(best_solution):
            best_solution = new_solution.copy()
            print(f'vns_algorithm: nadjeno je novo najbolje resenje: {best_solution.number_of_nodes()}')
    
    return best_solution

In [48]:
final_solution = vns_algorithm(G_global, 5, 10)
print("Broj cvorova poslednjeg grafa")
print(final_solution.number_of_nodes())


vns_algorithm: broj cvorova na pocetku: 78
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 73
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 73
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
73
Broj cvorova na izlasku iz local_search() 
41
vns_algorithm: nadjeno je novo najbolje resenje: 41
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 36
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
41
Broj cvorova na izlasku iz local_search() 
41
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 36
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
41
Broj cvorova na izlasku iz local_search() 
41
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 36
shakin

shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
50
Broj cvorova na izlasku iz local_search() 
50
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
27
Broj cvorova na izlasku iz local_search() 
27
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 22
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
27
Broj cvorova na izlasku iz local_search() 
27
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 48
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 43
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
43
Broj cvorova na izlasku iz local_search() 
43
shaking: broj cvorova pre pomeranj

Broj cvorova na izlasku iz local_search() 
37
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 42
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 19
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
vns_algorithm: nadjeno je novo najbolje resenje: 19
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 14
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 44
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 41
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
41
Broj cvorova na izlasku iz local_search() 
39
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 42
shak

shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
39
Broj cvorova na izlasku iz local_search() 
35
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 14
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 43
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 42
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
42
Broj cvorova na izlasku iz local_search() 
38
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 40
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 33
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
33
Broj cvorova na izlasku iz local_search() 
31
shaking: broj cvorova pre pomeranja

shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
39
Broj cvorova na izlasku iz local_search() 
39
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 38
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
38
Broj cvorova na izlasku iz local_search() 
37
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 15
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 14
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranj

...

Broj cvorova na izlasku iz local_search() 
31
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 43
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 42
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
42
Broj cvorova na izlasku iz local_search() 
39
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 15
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 39
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
39
Broj cvorova na izlasku iz local_search() 
39
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 42
shaking: broj cvorova posle pomeranja cvorova sa manje o

shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
42
Broj cvorova na izlasku iz local_search() 
41
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 36
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
36
Broj cvorova na izlasku iz local_search() 
35
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 38
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
38
Broj cvorova na izlasku iz local_search() 
37
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 42
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 36
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
36
Broj cvorova na izlasku iz local_search() 
31
shaking: broj cvorova pre pomeranja 

shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
39
Broj cvorova na izlasku iz local_search() 
39
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 37
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
37
Broj cvorova na izlasku iz local_search() 
37
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 42
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 39
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
39
Broj cvorova na izlasku iz local_search() 
34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 14
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranja

Broj cvorova na izlasku iz local_search() 
34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 40
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 35
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
35
Broj cvorova na izlasku iz local_search() 
34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 40
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 36
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
36
Broj cvorova na izlasku iz local_search() 
35
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 44
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 43
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
43
Broj cvorova na izlasku iz local_search() 
43
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 14
shaking: broj cvorova posle pomeranja cvorova sa manje od

Broj cvorova na izlasku iz local_search() 
34
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 43
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 42
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
42
Broj cvorova na izlasku iz local_search() 
36
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 14
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 0
shaking: modifikovano rešenje nije prihvatljivo
Broj cvorova na ulasku u local_search() 
19
Broj cvorova na izlasku iz local_search() 
19
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 41
shaking: broj cvorova posle pomeranja cvorova sa manje od k suseda: 40
shaking: modifikovano rešenje je prihvatljivo
Broj cvorova na ulasku u local_search() 
40
Broj cvorova na izlasku iz local_search() 
40
shaking: broj cvorova pre pomeranja cvorova sa manje od k suseda: 15
shaking: broj cvorova posle pomeranja cvorova sa manje o

In [43]:
from itertools import combinations

def brute_force_algorithm(G, k):
    all_nodes = list(G.nodes())
    n = len(all_nodes)
    best_solution = None
    best_size = float('inf')

    # Provera svih mogućih kombinacija čvorova
    for r in range(n, k-1, -1):
        for nodes in combinations(all_nodes, r):
            subgraph = G.subgraph(nodes)
            if nx.is_k_edge_connected(subgraph, k):
                if r < best_size:
                    best_solution = subgraph
                    best_size = r

    return best_solution